In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Scipy not supported!


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("i want to change my subscription", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
['casinotab', 'changesubscription', 'circledgame', 'goodbye', 'greeting', 'missingemails', 'playercontact', 'playerpaying', 'subscriptionmonth', 'thanks']


In [6]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Gavin\Documents\jupyter\TensorFlow Chat Bot\model.tflearn


In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [8]:
classify('i want to change my subscription')

[('changesubscription', 0.68732268), ('greeting', 0.2973344)]

In [9]:
response('i want to change my subscription')

On your dashboard there is a Upgrade/Renew Subscription button. You change your subscription there.


In [10]:
response('why is a game circled?')

All games are "Circled" when they are posted. As it gets closer to the start time the game becomes uncircled, usually 12 hours before tipoff/kickoff. Circled games have a circled game limit in Tools/Sports Settings. The idea is that the lines become more stable as it gets closer to the game. The circled game prevent sharps from making a big play on an early unstableline. You can however change the time when games are uncircled for various sports in Tools/Sports Settings and edit the individual sports at the bottom of the screen.


In [11]:
response('my player isn\'t getting emails')

If players are not receiving emails they should check their "Spam" or "Junk" mail folders to see if the emails have been wrongly flagged as spam. If they are, then marking these emails as "not spam" will hopefully prevent any more of these emails being wrongly filtered as spam.


In [12]:
response('Goodbye, see you later')

Have a nice day


**Ignore all below for now...**

Below code left in place as we can adapt if a context is to flow through a number of follow-up interactions in a conversation. This is where bot asks a question it wants more info for.

In [13]:
context

{}

In [20]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [21]:
# show context
context

{'123': 'rentalday'}

In [22]:
response('today')

For rentals today please call 1-800-MYMOPED


In [16]:
classify('today')

[('today', 0.5943121910095215)]

In [23]:
# clear context
response("Hi there!", show_details=True)

context: 
tag: greeting
Hi there, how can I help?


In [24]:
response('today')
classify('today')

[('today', 0.5943121910095215)]

In [25]:
response("thanks, your great")

Happy to help!
